# Uma introdução aos modelos de difusão

Um notebook do Google Colab que permite gerar imagens de textos usando modelos de [Stable Diffusion (por Stability AI, Runway & Compvis)](https://en.wikipedia.org/wiki/Stable_Diffusion).

Esse notebook é uma alternativa a interface WEBs enquanto oferece uma simples interface de usuário para iniciantes em Stable Diffusionç

<br/>

Baseado no repositório em [GitHub](https://github.com/redromnon/stable-diffusion-interactive-notebook).

In [ ]:
#@title 👇 Instalando dependências { display-mode: "form" }
#@markdown ---
#@markdown Selecione **GPU** como ambiente de executação:<br/>
#@markdown *Ambiente de execução->Alterar o tipo de ambiente de execução->Selecione uma GPU*
#@markdown
#@markdown ---

!pip -q install torch diffusers transformers accelerate scipy safetensors xformers mediapy ipywidgets==7.7.1

In [ ]:
#@title 👇 Selecione o modelo { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Selecione o modelo** - Uma lista de modelos Stable Diffusion.
#@markdown - **Selecione o agendador** - Uma lista de agendadores.
#@markdown - **Conteúdo censurado** - Ativar/desativar conteúdo censurado.
#@markdown
#@markdown ---

from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, DDIMScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers.models import AutoencoderKL
import torch
import ipywidgets as widgets
import importlib

#Enable third party widget support
from google.colab import output
output.enable_custom_widget_manager()

#Pipe
pipe = None

#Models
select_model = widgets.Dropdown(
    options=[
        ("Stable Diffusion 2.1 Base" , "stabilityai/stable-diffusion-2-1-base"),
        ("Stable Diffusion 2.1" , "stabilityai/stable-diffusion-2-1"),
        ("Stable Diffusion 1.5", "runwayml/stable-diffusion-v1-5"),
        ("Dreamlike Photoreal 2.0" , "dreamlike-art/dreamlike-photoreal-2.0"),
        ("OpenJourney v4" , "prompthero/openjourney-v4")
    ],
    description="Select Model:"
)

#Schedulers
select_sampler = widgets.Dropdown(
    options=[
        "EulerAncestralDiscreteScheduler",
        "EulerDiscreteScheduler",
        "UniPCMultistepScheduler",
        "DDIMScheduler"
    ],
    description="Select Schedular:"
)
select_sampler.style.description_width = "auto"

#Safety Checker
safety_check = widgets.Checkbox(
    value=True,
    description="Enable Safety Check",
    layout=widgets.Layout(margin="0px 0px 0px -85px")
)

#Output
out = widgets.Output()

#Apply Settings
apply_btn = widgets.Button(
    description="Apply",
    button_style="info"
)


#Get scheduler
def get_scheduler(name):

  match name:

    case "EulerAncestralDiscreteScheduler":
      return EulerAncestralDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "DDIMScheduler":
      return DDIMScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "EulerDiscreteScheduler":
      return EulerDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")

    case "UniPCMultistepScheduler":
      return UniPCMultistepScheduler.from_pretrained(select_model.value, subfolder="scheduler")

#Run pipeline
def pipeline(p):

  global pipe

  out.clear_output()
  apply_btn.disabled = True

  with out:

    print("Running, please wait...")

    pipe = StableDiffusionPipeline.from_pretrained(
      select_model.value,
      scheduler=get_scheduler(select_sampler.value),
      torch_dtype=torch.float16,
      vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
    ).to("cuda")

    if not safety_check.value:
      pipe.safety_checker = None

    pipe.enable_xformers_memory_efficient_attention()

    print("Finished!")

  apply_btn.disabled = False


#Display
apply_btn.on_click(pipeline)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Configure Pipeline</h2>"),
      select_model, select_sampler, safety_check, apply_btn, out
    ]
)


In [ ]:
#@title 👇Gerando imagens { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Prompt** - Descrição da imagem
#@markdown - **Prompt negativo** - Objetos para ignorar na geração da imagem
#@markdown - **Passos** - Quantidade de passos de desruído. Quanto maior a quantidade de passos, maior a qualidade e precisão da imagem.
#@markdown - **CFG** - Quanto mais restrita a IA é. Valores de 0 a20
#@markdown - **Seed** - Definir valor para resultados reproduzíveis. -1 para aleatório.
#@markdown ---
import ipywidgets as widgets, mediapy, random
import IPython.display


#PARAMETER WIDGETS
width = "300px"

prompt = widgets.Textarea(
    value="",
    placeholder="Enter prompt",
    #description="Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

neg_prompt = widgets.Textarea(
    value="",
    placeholder="Enter negative prompt",
    #description="Negative Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

num_images = widgets.IntText(
    value=1,
    description="Images:",
    layout=widgets.Layout(width=width),
)

steps = widgets.IntText(
    value=30,
    description="Steps:",
    layout=widgets.Layout(width=width)
)

CFG = widgets.FloatText(
    value=7.5,
    description="CFG:",
    layout=widgets.Layout(width=width)
)

img_height = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768)],
    value=512,
    description="Height:",
    layout=widgets.Layout(width=width)
)

img_width = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768)],
    value=512,
    description="Width:",
    layout=widgets.Layout(width=width)
)

random_seed = widgets.IntText(
    value=-1,
    description="Seed:",
    layout=widgets.Layout(width=width),
    disabled=False
)

generate = widgets.Button(
    description="Generate",
    disabled=False,
    button_style="primary"
)

display_imgs = widgets.Output()


#RUN
def generate_img(i):

  #Clear output
  display_imgs.clear_output()
  generate.disabled = True

  #Calculate seed
  seed = random.randint(0, 2147483647) if random_seed.value == -1 else random_seed.value

  with display_imgs:

    print("Running...")

    images = pipe(
      prompt.value,
      height = img_height.value,
      width = img_width.value,
      num_inference_steps = steps.value,
      guidance_scale = CFG.value,
      num_images_per_prompt = num_images.value,
      negative_prompt = neg_prompt.value,
      generator = torch.Generator("cuda").manual_seed(seed),
    ).images
    mediapy.show_images(images)

    print(f"Seed:\n{seed}")

  generate.disabled = False

#Display
generate.on_click(generate_img)

widgets.VBox(
    [
      widgets.AppLayout(
        header=widgets.HTML(
            value="<h2>Stable Diffusion</h2>",
        ),
        left_sidebar=widgets.VBox(
            [num_images, steps, CFG, img_height, img_width, random_seed]
        ),
        center=widgets.VBox(
            [prompt, neg_prompt, generate]
        ),
        right_sidebar=None,
        footer=None
      ),
      display_imgs
    ]
)